<a href="https://colab.research.google.com/github/Noelle-Pastor/Top-American-Authors-in-19th---21st-Century-Literary-Anthologies/blob/main/2_Loading_Queried_Data_for_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading data for plots**

In [ ]:
library(tidyverse)

In [ ]:
DIR_FILEPATH = ""

AUTHOR_STATS_FILEPATH = ""

### For Column Chart by Century - Author vs Page Count
Load one century of the author stats table, fields `Author`, `Num_Pages`, `Frequency`, and `Proportion`.

In [ ]:
author_stats <- read_delim(
  AUTHOR_STATS_FILEPATH,
  delim = "\t",
  quote = ""
) %>%
  filter(author != "On Steel") %>%
  arrange(desc(frequency), desc(total_num_pages)) %>%
  mutate(proportion = proportion*100) #%>%
  #head(30)



### Column Chart - Native American Literature vs Page Count
####Get `Num_Pages`, `Frequency`, and `Century` for Native American Literature.


Filter author stats for only Native American Literature and add `century` field. Run this code once for each century's  data. End with master table with a row for Native American Literature stats for each century.

In [ ]:
native_lit_stats <- read_delim(
  AUTHOR_STATS_FILEPATH,
  delim = "\t",
  quote = ""
) %>%
  filter(author == "Native American Literature") %>%
  mutate(century = "21st Century") %>%
  mutate(proportion = proportion*100)

In [ ]:
#a <- tibble()
a <- bind_rows(a, native_lit_stats)
native_lit_stats <- a

### Clustered Column Chart -- Page Count: Native American Literature vs  Most Frequent Author.
#### Grouped by century.

Combine previous output `native_lit_stats` with the same output for the most frequently occuring author `num_pages`, `frequency`, `century`. Run code for each century.

In [ ]:

# COMBINE native_lit_stats WTIH:


author_stats <- read_delim(
  AUTHOR_STATS_FILEPATH,
  delim = "\t",
  quote = ""
) %>%
  filter(author != "Native American Literature") %>%
  filter(frequency == max(frequency)) %>%
  mutate(proportion = proportion*100, century = "21st Century") %>%
  head(1)


In [ ]:
#b <- tibble()
b <- bind_rows(b, author_stats)
most_freq_auth_stats <- b

In [ ]:
combined_native_and_other_stats <- bind_rows(most_freq_auth_stats,native_lit_stats)

Ensure author appears correctly on plot.

In [ ]:
combined_native_and_other_stats$author <- forcats::fct_relevel(
  combined_native_and_other_stats$author,
  "Native American Literature"
)

### Get TOP 30 AUTHORS, over ALL CENTURIES, by TOTAL PAGES


In [ ]:
#LOAD ONE CENTURY OF THE author_stats QUERIED + SAVED TABLE
read_file <- function(filepath){

  author_stats <- read_delim(
    filepath,
    delim = "\t",
    quote = "") %>%
    arrange(desc(frequency), desc(total_num_pages))

  return(author_stats)
}


In [ ]:
#ADD STATS TO MASTER TABLE
get_full_tib <- function(dir_filepath){

  print("Getting files to combine...")
  all_files <- list.files(path=dir_filepath, full.names = TRUE) %>%
    sort()
  names_only <-list.files(path=dir_filepath, full.names = FALSE) %>%
    sort()
  print(paste("Files to add to master table: ", names_only))


  master_tib <- tibble()
  i=1
  for (filepath in all_files){
    print(paste("Adding ", names_only[i], "to master table..."))
    current_tib <- read_file(filepath) %>%
      mutate(century = substr(names_only[i], 0, 4)) #files  start with century i.e. "20th" or "19th"
    master_tib <- bind_rows(master_tib, current_tib)
    print(paste(names_only[i], " added to master table."))
    i <- i + 1
  }

  # author, frequency, total_num_pages, proportion, century
  print("Master Table complete.")
  return(master_tib)
}

In [ ]:
#GET TOP 30 AUTHORS BY TOTAL PAGES
author_stats_all_centuries <- get_full_tib(DIR_FILEPATH) %>%
  group_by(author) %>%
  summarize(total_num_pages = sum(total_num_pages),
            frequency = sum(frequency)) %>%
  mutate(proportion = 100*round(total_num_pages/sum(total_num_pages), 4)) %>%
  arrange(desc(total_num_pages)) %>%
  head(30)